# Data science in finance, ex #4 
Eitan Rosenfelder

In [88]:
#organizing the data
import numpy as np
import pandas as pd
import re

#Data Source
import yfinance as yf

#brakes
from time import sleep
import random


In [4]:
#collecting the data
def collect(tick):
    data = yf.download(tickers=tick, period='5y', interval='1d')
    return data

In [7]:
#making data frames of rolling means
def sma(df, lst):    
    for i in lst:
        j = str(i)
        dat_length = str('Adj Close' + j)
        n = int(i)
        df = df
        df[dat_length] = data['Adj Close'].rolling(window=n).mean()
    return df

,Adj Close7,Adj Close14,Adj Close21,Adj Close30
Date,,,,
2016-01-05,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN
2016-01-11,NaN,NaN,NaN,NaN
...,...,...,...,...
2020-12-28,175.247144,174.654287,173.885239,173.573783
2020-12-29,174.847146,174.753573,173.740478,173.757657
2020-12-30,174.517147,174.669288,173.806192,173.815694


In [10]:
# Create a dataframe of all adj close prices
def compare(data, sma_short,sma_long, Column_s, column_l):
    udata = pd.DataFrame(index=data.index)
    udata['stock_adj_close'] = data['Adj Close']
    udata[Column_s] = sma_short[Column_s]
    udata[column_l]= sma_long[column_l]
    return udata

In [58]:
#Create a Function to signal when to buy and sell
def buy_sell_fun(data, Column_s, column_l):
    sig_price_buy = []
    sig_price_sell = []
    flag = -1
    money = float(100000)
    x = float(0)

    for i in range(len(data)):
        if data[Column_s][i] > data[column_l][i]:
            if flag != 1:
                x = money/data['stock_adj_close'][i]
                sig_price_buy.append(x)
                sig_price_sell.append(np.nan)
                flag = 1
            else:
                sig_price_buy.append(np.nan)
                sig_price_sell.append(np.nan)
        elif data[Column_s][i] < data[column_l][i]:
            if flag !=0:
                money = x*data['stock_adj_close'][i]
                sig_price_buy.append(np.nan)
                sig_price_sell.append(money)
                x = 0
                flag = 0
            else:
                sig_price_buy.append(np.nan)
                sig_price_sell.append(np.nan)
        else:
            sig_price_buy.append(np.nan)
            sig_price_sell.append(np.nan)

    return (sig_price_buy, sig_price_sell, money, x)

def chart_data(data,stock, Column_s, column_l):
    #store the buy and sell data into a variable
    buy_sell = buy_sell_fun(data, Column_s, column_l)
    data['buy_signal_price'] = buy_sell[0]
    data['sell_signal_price']=buy_sell[1]

    #Looking for the dates of the buy signals
    buy_dates = data.loc[data['buy_signal_price']>0]

    #Looking for the dates of the sell signals
    sell_dates = data.loc[data['sell_signal_price']>0]

    #Dropping uneccessary columns
    buy_dates.drop(columns = ['sell_signal_price', Column_s, column_l], inplace = True)
    sell_dates.drop(columns = ['buy_signal_price', Column_s, column_l], inplace = True)

    # DF of dates with prices
    dates = pd.concat([buy_dates, sell_dates],sort = True)

    # Now to organize by the dates, drop adj_close price column, and make date the index
    dates.sort_values(by = 'Date', inplace = True)
    dates.drop(columns = ['stock_adj_close'],inplace = True)
    dates.reset_index('Date', inplace = True)



In [89]:
def measuring_performance(data, sma_short, sma_long, stock):
    results = pd.DataFrame(columns =  ["stock", "length short", "length long", "cash"])
    for column_l in sma_long:
        for Column_s in sma_short:
            udata = compare(data, sma_short, sma_long, Column_s, column_l)
            sig_price_buy, sig_price_sell, money, x = buy_sell_fun(udata, Column_s, column_l)
            chart_data(udata,stock, Column_s, column_l)
            udata.index = np.arange(1, udata.shape[0] + 1)
            last = pd.DataFrame(udata.apply(pd.Series.last_valid_index))
            date_buy = last.iloc[3][0]
            date_sell = last.iloc[4][0]
            
            #checking whether the last action was buy or sell
            if date_buy>date_sell:
                final = money
            else:
                final = x
                
            i = len(results) + 1
            short = re.sub("[^\d\.]", "", Column_s)
            long = re.sub("[^\d\.]", "", column_l)
            results.loc[i] = [stock, short, long, final]
    return results

In [90]:
#finding the best combination for each stock
def best_combination(list_dow, lst_short, lst_long):
    best = pd.DataFrame(columns =  ["stock", "length short", "length long", "cash"])
    for i in list_dow:
        sma_short = pd.DataFrame()
        sma_long = pd.DataFrame()
        sma_short = sma(sma_short, lst_short)
        sma_long = sma(sma_long, lst_long)
        sleep(random.randint(5, 10))
        data = collect(i)
        order = measuring_performance(data, sma_short, sma_long, i)
        ans = order.nlargest(1, "cash")
        best = pd.concat([best,ans], axis=0,ignore_index=True )
    return best

In [91]:
#the list of stocks in the dow
list_dow = ["MMM","AXP","AMGN","AAPL","BA","CAT","CVX","CSCO","KO","DOW","GS","HD","HON","IBM","INTC","JNJ","JPM","MCD","MRK","MSFT","NKE","PG","CRM","TRV","UNH","VZ","V","WBA","WMT","DIS"]
#list of lengths for short and long
lst_short = [7,14,21,30]
lst_long = [50,100,150,200]
#calling the function of measuring
best = best_combination(list_dow, lst_short, lst_long)
best

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,stock,which short,which long,cash
0,MMM,21,50,112524.316070
1,AXP,21,50,183212.707666
2,AMGN,21,50,196299.141751
3,AAPL,21,50,421908.144962
4,BA,21,50,221111.367086
5,CAT,30,50,213562.961640
6,CVX,30,200,101720.149930
7,CSCO,7,100,184701.218512
8,KO,30,150,138043.306916
9,DOW,14,50,127138.938332
